## **RADI608: Data Mining and Machine Learning**

### Assignment: Genetic Algorithm
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Suppose $x_i$ is a decision variable where $x$ is a real number at $i = [1, 2, 3, 4, 5, 6]$. We first initialized a population that consists of eight (8) chromosomes with 6 genes each. We encode the decision variable to binary chromosomes. This is represented in the figure below.

<center>
<img src = "../figures/01.jpg" width = '250'> <br>
</center>

We need to decode the bitstrings to numbers prior to evaluating each with the objective function. We can achieve this by first decoding each substring to an integer, then scaling the integer to the desired range. This will give a vector of values in the range that can then be provided to the objective function for evaluation.

<center>
<img src = "../figures/02.jpg" width = '900'> <br>
</center>

Parents are used as the basis for generating the next generation of candidate points and one parent for each position in the population is required. Parents are then taken in pairs and used to create two children. Recombination is performed using a crossover operator. This involves selecting a random split point on the bit string, then creating a child with the bits up to the split point from the first parent and from the split point to the end of the string from the second parent. This process is then inverted for the second child.

<center>
<img src = "../figures/03.jpg" width = '1200'> <br>
</center>

In [7]:
import sys
import numpy as np
import GA as ga
import random 
import matplotlib.pyplot as plt

equation_inputs = [4, -2, 7, -5, 11, 1]
num_weights = len(equation_inputs)
chromosomes = 8
mating = 4

random.seed(413)
new_population = np.random.uniform(low = -5.0, high = 5.0, size = (chromosomes, num_weights))


In [5]:
def cal_fitness(inputs, initial_population):
    ''' 
    We changed the cal_pop_fitness into cal_fitness
    This follows the equation: 
        y = w1 * (x1 ** 2) + w2 * (x2 ** 3) + (w3 * x3) + (w4 * x4) + (w5 * x5) + (w6 * x6)
    The equation has six inputs and six weights
    '''
    fitness = inputs[0] * (initial_population[:, 0] ** 2)   + \
              inputs[1] * (initial_population[:, 1] ** 3)   + \
              inputs[2] * (initial_population[:, 2])        + \
              inputs[3] * (initial_population[:, 3])        + \
              inputs[4] * (initial_population[:, 4])        + \
              inputs[5] * (initial_population[:, 5])
    return fitness

def crossover(parents, offspring_size, crossover_rate = 0.1):
    offspring = np.empty(offspring_size)
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]

        random.seed(413)
        crossover_prob = np.random.uniform(0.0, 0.8, 1)

        if crossover_prob < crossover_rate:
            offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
            offspring[k, crossover_point: ] = parents[parent2_idx, crossover_point: ]
    return offspring

def mutation(offspring_crossover, mutations_rate):
    mutations_counter = np.uint8(offspring_crossover.shape[1])
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1

        random.seed(413)
        random_value  = np.random.uniform(-1.0, 1.0, 1)
        mutation_prob = np.random.uniform(-1.0, 1.0, 1)

        if mutation_prob < mutations_rate:
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover

def main_genetic(equation_inputs, new_population, chromosomes, mating):
    num_weights = len(equation_inputs)
    sol_per_pop = chromosomes
    num_parents_mating = mating
    pop_size = (sol_per_pop, num_parents_mating)

    best_outputs = []
    num_generations = 1

    crossover_rate = 0.8 
    mutations_rate = 0.1

    for generation in range(num_generations):
        fitness = cal_fitness(equation_inputs, new_population)
        best_outputs.append(np.max(fitness))

        parents = ga.select_mating_pool(new_population, fitness, num_parents_mating)

        offspring_size = (pop_size[0] - parents.shape[0], num_weights)
        offspring_crossover = crossover(parents, 
                                        offspring_size = offspring_size, 
                                        crossover_rate = crossover_rate)

        offspring_mutation  = mutation(offspring_crossover, 
                                       mutations_rate = mutations_rate)

        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:,  :] = offspring_mutation

    fitness = cal_fitness(equation_inputs, new_population)
    best_match_idx = np.where(fitness == np.max(fitness))
    print('Best approximate weight solution:  \t', np.round(new_population[best_match_idx, :], 2)[0])
    print('Best approximate solution fitness: \t', np.round(fitness[best_match_idx][0], 3))
    return best_outputs

best_outputs = main_genetic(equation_inputs, new_population, chromosomes, mating)

Best approximate weight solution:  	 [[-4.88 -0.65  4.07 -0.32  1.25  0.7 ]]
Best approximate solution fitness: 	 140.162


In [30]:
def scorer(w, equation_inputs):
    return (w * (equation_inputs[0] ** 2)) + (w * (equation_inputs[1] ** 3)) + (w * equation_inputs[2]) + (w * equation_inputs[3]) + (w * equation_inputs[4]) + (w * equation_inputs[5])

equation_inputs = [4, -2, 7, -5, 11, 1]
decoded = [1, 5, 30, 7, 16, 15, 46, 4]

scores = []
for decode in decoded:
    scores.append(scorer(decode, equation_inputs))
scores

[22, 110, 660, 154, 352, 330, 1012, 88]

In [31]:
for_selection = scores.copy()
selection_by_parents = []
for idx in for_selection:
    sum_value = sum(for_selection)
    selection_by_parents.append(np.round(idx/sum_value, 4))
selection_by_parents

[0.0081, 0.0403, 0.2419, 0.0565, 0.129, 0.121, 0.371, 0.0323]

In [34]:
selection = []
for select in range(len(selection_by_parents)):
    selection.append(sum(selection_by_parents[0:select]))
selection.append(1)
selection

[0, 0.0081, 0.0484, 0.2903, 0.3468, 0.4758, 0.5968, 0.9678, 1]

In [2]:
from numpy.random import randint
from numpy.random import rand
 

def objective(x):
	return x[0]**2.0 + x[1]**.0
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded
 
def selection(pop, scores, k = 3):
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]
 
def crossover(p1, p2, r_cross):
	c1, c2 = p1.copy(), p2.copy()
	if rand() < r_cross:
		pt = randint(1, len(p1)-2)
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]
 
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		if rand() < r_mut:
			bitstring[i] = 1 - bitstring[i]
 
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# # pop = [randint(0, 2, 6).tolist() for _ in range(8)]
	pop =  [[0, 0, 0, 0, 0, 1],
			[0, 0, 0, 1, 0, 1],
			[0, 1, 1, 1, 1, 0],
			[0, 0, 0, 1, 1, 1],
			[0, 1, 0, 0, 0, 0],
			[0, 0, 1, 1, 1, 1],
			[1, 0, 1, 1, 1, 1],
			[0, 0, 0, 1, 0, 0]]
	print(decode(bounds, n_bits, pop[0]))
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	for gen in range(n_iter):
		decoded = [decode(bounds, n_bits, p) for p in pop]
		scores = [objective(d) for d in decoded]

		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print("Iteration \t %d, New best f(%s) = %f" % (gen,  decoded[i], scores[i]))

		selected = [selection(pop, scores) for _ in range(n_pop)]

		children = list()
		for i in range(0, n_pop, 2):
			p1, p2 = selected[i], selected[i+1]
			for c in crossover(p1, p2, r_cross):
				mutation(c, r_mut)
				children.append(c)
		pop = children
	return [best, best_eval]
 
# # define range for input
# bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# # define the total iterations
# n_iter = 100
# # bits per variable
# n_bits = 6
# n_pop = 8
# r_cross = 0.8
# r_mut = 0.1
# best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
# print('Done!')
# decoded = decode(bounds, n_bits, best)
# print('f(%s) = %f' % (decoded, score))

In [11]:
initial_population = [randint(0, 2, 6).tolist() for _ in range(8)]
initial_population

[[0, 1, 0, 0, 1, 1],
 [0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 0],
 [1, 0, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 0]]

In [4]:
pop =  [[0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 1],
        [0, 1, 1, 1, 1, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 0, 0]]

pop

[[0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 0]]